# Проверка базовых простейщих моделей на корпусе crowdsource_for_grant

In [1]:
import sys
import os
source_path = os.path.split(os.path.abspath(""))[0]
sys.path.append(source_path)

In [2]:
import numpy as np
import pandas as pd

from sklearn.dummy import DummyClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

from datasets import load_dataset

import wandb

from src.data.evaluate import evaluate_all_tasks

## Данные

In [3]:
inp_tr = load_dataset("sagteam/author_profiling", split="train").to_pandas()
inp_vl = load_dataset("sagteam/author_profiling", split="validation").to_pandas()
inp_ts = load_dataset("sagteam/author_profiling", split="test").to_pandas()

inp_tr = inp_tr.replace("None", None)
inp_vl = inp_vl.replace("None", None)
inp_ts = inp_ts.replace("None", None)

No config specified, defaulting to: author_profiling/main
Reusing dataset author_profiling (/home/saguser/.cache/huggingface/datasets/sagteam___author_profiling/main/2.0.1/6c07c7d8f4349ec5178d8f0c93b529c36bd3a2c3cb31504a8e8756fd34bbf064)
No config specified, defaulting to: author_profiling/main
Reusing dataset author_profiling (/home/saguser/.cache/huggingface/datasets/sagteam___author_profiling/main/2.0.1/6c07c7d8f4349ec5178d8f0c93b529c36bd3a2c3cb31504a8e8756fd34bbf064)
No config specified, defaulting to: author_profiling/main
Reusing dataset author_profiling (/home/saguser/.cache/huggingface/datasets/sagteam___author_profiling/main/2.0.1/6c07c7d8f4349ec5178d8f0c93b529c36bd3a2c3cb31504a8e8756fd34bbf064)


In [4]:
x_tr = inp_tr["text"].values
x_vl = inp_vl["text"].values
x_ts = inp_ts["text"].values

In [5]:
l_columns = ["age_group", "gender", "no_imitation", "age_imitation",
                 "gender_imitation", "style_imitation"]

In [6]:
# wandb_project = "deb-GraphHyperOpt"
wandb_project = None

## Модели

### Dummy MostFrequent

In [7]:
res_dummy_most_freq = {"id": inp_ts["id"], "author_id": inp_ts["author_id"]}
for task_name in l_columns:
    
    df_task_tr = inp_tr[inp_tr[task_name].notnull()]
    df_task_vl = inp_vl[inp_vl[task_name].notnull()]
    x_task_tr = df_task_tr["text"].values
    x_task_vl = df_task_vl["text"].values
    y_task_tr = df_task_tr[task_name].values
    y_task_vl = df_task_vl[task_name].values
    
    model = DummyClassifier(strategy="most_frequent")
    model.fit(x_task_tr, y_task_tr)
    pred = model.predict(x_ts)
    res_dummy_most_freq[task_name] = pred

d_model_config = {"model_type": "Dummy", "stragegy": "most_frequent"}
df_res_dummy_most_freq = pd.DataFrame(res_dummy_most_freq)
d_scores_dummy_most_freq = evaluate_all_tasks(inp_ts, df_res_dummy_most_freq, wandb_project=wandb_project, wandb_config=d_model_config)

/home/saguser/anaconda3/envs/graph_evalution_model/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/saguser/anaconda3/envs/graph_evalution_model/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/saguser/anaconda3/envs/graph_evalution_model/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

In [8]:
print("F1-score weihted, average: {0:0.2f}".format(d_scores_dummy_most_freq["average"]["all"]["f1_weighted"]))
print("")
print("Detail by tasks")
for k, v in d_scores_dummy_most_freq["details"].items():
    print("Task: {0}".format(k))
    print(v["all"]["classification_report"])
    print("")

F1-score weihted, average: 0.40

Detail by tasks
Task: age_group
              precision    recall  f1-score   support

        0-19       0.00      0.00      0.00       195
       20-29       0.44      1.00      0.61      1131
       30-39       0.00      0.00      0.00       683
       40-49       0.00      0.00      0.00       351
         50+       0.00      0.00      0.00       204

    accuracy                           0.44      2564
   macro avg       0.09      0.20      0.12      2564
weighted avg       0.19      0.44      0.27      2564


Task: gender
              precision    recall  f1-score   support

      female       0.50      1.00      0.67      1290
        male       0.00      0.00      0.00      1274

    accuracy                           0.50      2564
   macro avg       0.25      0.50      0.33      2564
weighted avg       0.25      0.50      0.34      2564


Task: no_imitation
                    precision    recall  f1-score   support

  no_any_imitation      

### Dummy random

In [9]:
res_dummy_random = {"id": inp_ts["id"], "author_id": inp_ts["author_id"]}
for task_name in l_columns:
    
    df_task_tr = inp_tr[inp_tr[task_name].notnull()]
    df_task_vl = inp_vl[inp_vl[task_name].notnull()]
    x_task_tr = df_task_tr["text"].values
    x_task_vl = df_task_vl["text"].values
    y_task_tr = df_task_tr[task_name].astype(str).values
    y_task_vl = df_task_vl[task_name].astype(str).values
    
    model = DummyClassifier(strategy="uniform")
    model.fit(x_task_tr, y_task_tr)
    pred = model.predict(x_ts)
    res_dummy_random[task_name] = pred

d_model_config = {"model_type": "Dummy", "stragegy": "uniform"}
df_res_dummy_random = pd.DataFrame(res_dummy_random)
d_scores_dummy_random = evaluate_all_tasks(inp_ts, df_res_dummy_random, wandb_project=wandb_project, wandb_config=d_model_config)
d_scores_dummy_random["average"]

/home/saguser/anaconda3/envs/graph_evalution_model/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/saguser/anaconda3/envs/graph_evalution_model/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/saguser/anaconda3/envs/graph_evalution_model/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

{'all': {'f1_weighted': 0.43667098646959346,
  'f1_micro': 0.4259400436205218,
  'f1_macro': 0.41480144673178687},
 'no_im': {'f1_weighted': 0.5467888735499637,
  'f1_micro': 0.4325628170911247,
  'f1_macro': 0.31552485858004414},
 'with_any_im': {'f1_weighted': 0.46386765848004335,
  'f1_micro': 0.42263959031648346,
  'f1_macro': 0.3778657728208837}}

In [10]:
print("F1-score weihted, average: {0:0.2f}".format(d_scores_dummy_random["average"]["all"]["f1_weighted"]))
print("")
print("Detail by tasks")
for k, v in d_scores_dummy_random["details"].items():
    print("Task: {0}".format(k))
    print(v["all"]["classification_report"])
    print("")

F1-score weihted, average: 0.44

Detail by tasks
Task: age_group
              precision    recall  f1-score   support

        0-19       0.08      0.20      0.11       195
       20-29       0.44      0.21      0.28      1131
       30-39       0.26      0.18      0.21       683
       40-49       0.16      0.24      0.20       351
         50+       0.07      0.18      0.10       204

    accuracy                           0.20      2564
   macro avg       0.20      0.20      0.18      2564
weighted avg       0.30      0.20      0.22      2564


Task: gender
              precision    recall  f1-score   support

      female       0.52      0.52      0.52      1290
        male       0.51      0.51      0.51      1274

    accuracy                           0.52      2564
   macro avg       0.52      0.52      0.52      2564
weighted avg       0.52      0.52      0.52      2564


Task: no_imitation
                    precision    recall  f1-score   support

  no_any_imitation      

### Dummy stratified

In [11]:
res_dummy_stratified = {"id": inp_ts["id"], "author_id": inp_ts["author_id"]}
for task_name in l_columns:
    
    df_task_tr = inp_tr[inp_tr[task_name].notnull()]
    df_task_vl = inp_vl[inp_vl[task_name].notnull()]
    x_task_tr = df_task_tr["text"].values
    x_task_vl = df_task_vl["text"].values
    y_task_tr = df_task_tr[task_name].astype(str).values
    y_task_vl = df_task_vl[task_name].astype(str).values
    
    model = DummyClassifier(strategy="stratified")
    model.fit(x_task_tr, y_task_tr)
    pred = model.predict(x_ts)
    res_dummy_stratified[task_name] = pred

d_model_config = {"model_type": "Dummy", "stragegy": "stratified"}
df_res_dummy_stratified = pd.DataFrame(res_dummy_stratified)
d_scores_dummy_stratified = evaluate_all_tasks(inp_ts, df_res_dummy_stratified, wandb_project=wandb_project, wandb_config=d_model_config)
d_scores_dummy_stratified["average"]

/home/saguser/anaconda3/envs/graph_evalution_model/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/saguser/anaconda3/envs/graph_evalution_model/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/saguser/anaconda3/envs/graph_evalution_model/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

{'all': {'f1_weighted': 0.46718502354412433,
  'f1_micro': 0.4684491669001109,
  'f1_macro': 0.4219433326689781},
 'no_im': {'f1_weighted': 0.566000639299547,
  'f1_micro': 0.4687720447402713,
  'f1_macro': 0.31832037285613407},
 'with_any_im': {'f1_weighted': 0.4954020379936445,
  'f1_micro': 0.4683357050609551,
  'f1_macro': 0.3896042981390428}}

In [12]:
print("F1-score weihted, average: {0:0.2f}".format(d_scores_dummy_stratified["average"]["all"]["f1_weighted"]))
print("")
print("Detail by tasks")
for k, v in d_scores_dummy_stratified["details"].items():
    print("Task: {0}".format(k))
    print(v["all"]["classification_report"])
    print("")

F1-score weihted, average: 0.47

Detail by tasks
Task: age_group
              precision    recall  f1-score   support

        0-19       0.07      0.08      0.07       195
       20-29       0.45      0.45      0.45      1131
       30-39       0.26      0.27      0.27       683
       40-49       0.15      0.13      0.14       351
         50+       0.08      0.06      0.07       204

    accuracy                           0.30      2564
   macro avg       0.20      0.20      0.20      2564
weighted avg       0.30      0.30      0.30      2564


Task: gender
              precision    recall  f1-score   support

      female       0.50      0.51      0.50      1290
        male       0.49      0.49      0.49      1274

    accuracy                           0.50      2564
   macro avg       0.50      0.50      0.50      2564
weighted avg       0.50      0.50      0.50      2564


Task: no_imitation
                    precision    recall  f1-score   support

  no_any_imitation      

### LinearSVC, TF-IDF н-граммы символов

In [13]:
res_svm_tfidf = {"id": inp_ts["id"], "author_id": inp_ts["author_id"]}
d_model_config = {"model_type": "LinearSVC_TFIDF", "tfidf":  {"analyzer": "char", "ngram_range": (3, 8), "min_df": 3,  "max_features": 1000}}
for task_name in l_columns:
    df_task_tr = inp_tr[inp_tr[task_name].notnull()]
    x_task_tr = df_task_tr["text"].values
    y_task_tr = df_task_tr[task_name].astype(str).values
    
    vect = TfidfVectorizer(**d_model_config["tfidf"])
    vect.fit(x_task_tr)
    x_task_tr_v = vect.transform(x_task_tr)
    x_ts_v = vect.transform(x_ts)
    model = LinearSVC()
    model.fit(x_task_tr_v, y_task_tr)
    pred = model.predict(x_ts_v)
    res_svm_tfidf[task_name] = pred
    
df_res_svm_tfidf = pd.DataFrame(res_svm_tfidf)
d_scores_svm_tfidf = evaluate_all_tasks(inp_ts, df_res_svm_tfidf, wandb_project=wandb_project, wandb_config=d_model_config)
d_scores_svm_tfidf["average"]

# wandb.finish()

/home/saguser/anaconda3/envs/graph_evalution_model/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/saguser/anaconda3/envs/graph_evalution_model/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/saguser/anaconda3/envs/graph_evalution_model/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

{'all': {'f1_weighted': 0.6324242559831729,
  'f1_micro': 0.6501372284443945,
  'f1_macro': 0.5835935496410375},
 'no_im': {'f1_weighted': 0.7321493035701009,
  'f1_micro': 0.6722024714309791,
  'f1_macro': 0.41248800563493454},
 'with_any_im': {'f1_weighted': 0.6468557705307492,
  'f1_micro': 0.6391908377275866,
  'f1_macro': 0.5040100494383989}}

In [14]:
print("F1-score weihted, average: {0:0.2f}".format(d_scores_svm_tfidf["average"]["all"]["f1_weighted"]))
print("")
print("Detail by tasks")
for k, v in d_scores_svm_tfidf["details"].items():
    print("Task: {0}".format(k))
    print(v["all"]["classification_report"])
    print("")

F1-score weihted, average: 0.63

Detail by tasks
Task: age_group
              precision    recall  f1-score   support

        0-19       0.21      0.04      0.06       195
       20-29       0.48      0.74      0.58      1131
       30-39       0.31      0.28      0.29       683
       40-49       0.23      0.08      0.12       351
         50+       0.33      0.07      0.11       204

    accuracy                           0.42      2564
   macro avg       0.31      0.24      0.23      2564
weighted avg       0.37      0.42      0.37      2564


Task: gender
              precision    recall  f1-score   support

      female       0.67      0.69      0.68      1290
        male       0.68      0.66      0.67      1274

    accuracy                           0.67      2564
   macro avg       0.67      0.67      0.67      2564
weighted avg       0.67      0.67      0.67      2564


Task: no_imitation
                    precision    recall  f1-score   support

  no_any_imitation      

In [15]:
# for k, v in d_scores_svm_tfidf["details"].items():
#     print(k)
#     print(v["all"]["f1_weighted"])
#     print(v["no_im"]["f1_weighted"])
#     print(v["with_any_im"]["f1_weighted"])
#     print("")

In [16]:
# import wandb
# wandb.finish()

In [17]:
print("Выполнено")

Выполнено
